In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install albumentations

In [ ]:
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import cv2
from PIL import Image
import random

base_dir = "/content/drive/MyDrive/project_lymphoma/SSL"

for subdir in ["images", "masks", "augmentation/images", "augmentation/masks"]:
    os.makedirs(f"{base_dir}/{subdir}", exist_ok=True)

npy_files_path = []
for dirpath, _, filenames in os.walk('/content/drive/MyDrive/project_lymphoma/lynsec/lynsec 3'):
    for filename in filenames:
        if filename.endswith('.npy'):
            npy_files_path.append(os.path.join(dirpath, filename))

for f in npy_files_path:
    data = np.load(f)
    base_name = os.path.splitext(os.path.basename(f))[0]


    original_image = data[..., :3].astype(np.uint8)
    mask_channel = data[..., 4]
    original_mask = (mask_channel == 1).astype(np.uint8) * 255

    Image.fromarray(original_image).save(f"{base_dir}/images/{base_name}.png")
    Image.fromarray(original_mask).save(f"{base_dir}/masks/{base_name}_mask.png")


    for aug_idx in range(10):

        transform = A.ReplayCompose([A.RandomCrop(width=256, height=256, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ColorJitter(brightness=random.uniform(0.01, 0.3), contrast=random.uniform(0.1, 0.03), saturation=random.uniform(0.01, 0.4), hue=random.uniform(0.01, 0.1), p=0.5),
            A.GaussianBlur(blur_limit=(3, 3), sigma_limit=(random.uniform(0.025, 0.1), 0.5), p=0.5),
        ])

        augmented = transform(image=original_image, mask=original_mask)
        aug_image = augmented["image"].astype(np.uint8)
        aug_mask = augmented["mask"].astype(np.uint8)


        replay = augmented["replay"]
        applied = [t["__class_fullname__"].split(".")[-1] for t in replay["transforms"] if t["applied"]]

        suffix = "_".join(applied) if applied else "no_aug"


        image_subdir = f"{base_dir}/augmentation/images/{base_name}"
        mask_subdir = f"{base_dir}/augmentation/masks/{base_name}"
        os.makedirs(image_subdir, exist_ok=True)
        os.makedirs(mask_subdir, exist_ok=True)

        aug_image_name = f"{base_name}_aug{aug_idx}_{suffix}.png"
        aug_mask_name = f"{base_name}_aug{aug_idx}_{suffix}_mask.png"

        Image.fromarray(aug_image).save(os.path.join(image_subdir, aug_image_name))
        Image.fromarray(aug_mask).save(os.path.join(mask_subdir, aug_mask_name))
